In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
from dotenv import load_dotenv, find_dotenv
from llama_index.core import SimpleDirectoryReader
import pandas as pd

#load environment variables from .env file
_ = load_dotenv

In [5]:
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=["../../data/Pmg_lds.md", "../../data/handbook.md"]).load_data()
node_parser = SentenceSplitter(chunk_size=2048, chunk_overlap=100)
nodes = node_parser.get_nodes_from_documents(documents)
# print(nodes[38])
print(len(nodes))

81


In [6]:
import os
from llama_index.core import SimpleDirectoryReader

# 1. Print your current working directory so you know where paths are relative to
print("CWD:", os.getcwd())

# 2. Load your documents
docs = SimpleDirectoryReader(
    input_files=["../../data/Pmg_lds.md", "../../data/handbook.md"]
).load_data()

# 3. Confirm how many Document objects you got
print(f"Loaded {len(docs)} documents")

# 4. For each one, print its file path (if available) and the length of its text
for i, doc in enumerate(docs):
    path = doc.extra_info.get("file_path", f"doc#{i}")
    text = doc.text or ""
    print(f"Document {i}: {path} → {len(text)} characters")
    # Optional: peek at the first/last 100 chars
    print("  start:", text[:100].replace("\n", " "))
    print("  end:  ", text[-100:].replace("\n", " "))


CWD: /home/promise/projects/CSE499/Agentic_rag/Notebooks/qa_pairs
Loaded 2 documents
Document 0: ../../data/Pmg_lds.md → 550833 characters
  start:  ## Preach My Gospel (D&C 50:14) (Page 5)  | Chapters | First Presidency Message | page no | | --- |
  end:   and will strengthen their faith in the Savior. - Love the Lord and serve Him the very best you can. 
Document 1: ../../data/handbook.md → 64640 characters
  start: # Message from the First Presidency and Quorum of the Twelve Apostles  Dear Fellow Missionary:  Afte
  end:   on, and when they taught, they taught with power and authority of God” (Alma 17:1–3; italics added).


/tmp/ipykernel_175806/667638478.py:17: DeprecationWarning: Call to deprecated function (or staticmethod) extra_info. ('extra_info' is deprecated, use 'metadata' instead.) -- Deprecated since version 0.12.2.
  path = doc.extra_info.get("file_path", f"doc#{i}")


In [7]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

# 1) Confirm where we are and load the docs
print("CWD:", os.getcwd())
docs = SimpleDirectoryReader(
    input_files=[
        "../../data/Pmg_lds.md",
        "../../data/handbook.md"
    ]
).load_data()
print(f"Loaded {len(docs)} documents\n")

# 2) Print per‑doc character counts
for i, doc in enumerate(docs):
    text = doc.text or ""
    print(f"Document {i}: {len(text):,} characters")
print()

# 3) Split each doc separately, count nodes per doc
chunk_size = 2048
chunk_overlap = 100
node_parser = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

for i, doc in enumerate(docs):
    nodes_i = node_parser.get_nodes_from_documents([doc])
    print(f"Doc {i} produced {len(nodes_i)} nodes (chunk_size={chunk_size}, overlap={chunk_overlap})")

print()

# 4) And finally split them together, total node count
nodes = node_parser.get_nodes_from_documents(docs)
print(f"Combined total nodes: {len(nodes)}\n")

# 5) Peek at the first few nodes
print("First 3 node snippets:")
for n in nodes[:3]:
    # show chars of each node
    print("—", repr(n.text[:]))
print("-", repr(nodes[-1].text[:]))

CWD: /home/promise/projects/CSE499/Agentic_rag/Notebooks/qa_pairs
Loaded 2 documents

Document 0: 550,833 characters
Document 1: 64,640 characters

Doc 0 produced 73 nodes (chunk_size=2048, overlap=100)
Doc 1 produced 8 nodes (chunk_size=2048, overlap=100)

Combined total nodes: 81

First 3 node snippets:
— '## Preach My Gospel (D&C 50:14) (Page 5)\n\n| Chapters | First Presidency Message | page no |\n| --- | --- | --- |\n|  | Introduction: How Can I Best Use Preach My Gospel? | vii |\n| 1 | What Is My Purpose as a Missionary? | 1 |\n| 2 | How Do I Study Effectively and Prepare to Teach? | 17 |\n| 3 | What Do I Study and Teach? | 29 |\n|  | <li>Lesson 1: The Message of the Restoration of the Gospel of Jesus Christ</li> | 31 |\n|  | <li>Lesson 2: The Plan of Salvation</li> | 47 |\n|  | <li>Lesson 3: The Gospel of Jesus Christ</li> | 60 |\n|  | <li>Lesson 4: The Commandments</l1? | 71 |\n|  | Lesson 5: Laws and Ordinances | 82 |\n| 4 | How Do I Recognize and Understand the Spirit? | 89 |

### qa with Ragas


In [9]:
# %pip install --upgrade "huggingface_hub>=0.20.0"

from ragas.testset import TestsetGenerator
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# generator with openai models
generator_llm = OpenAI(model="gpt-4o")
embeddings = OpenAIEmbedding(model="text-embedding-3-large")

generator = TestsetGenerator.from_llama_index(
    llm=generator_llm,
    embedding_model=embeddings,
)

In [10]:
# generate testset
testset = generator.generate_with_llamaindex_docs(
    documents=nodes,
    testset_size=200,
)

Applying HeadlinesExtractor:   0%|          | 0/79 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/81 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/84 [00:00<?, ?it/s]

Property 'summary' already exists in node '764564'. Skipping!
Property 'summary' already exists in node '2f36fe'. Skipping!
Property 'summary' already exists in node 'ace78b'. Skipping!
Property 'summary' already exists in node 'e0abd2'. Skipping!
Property 'summary' already exists in node '6bbf1e'. Skipping!


...

Property 'summary_embedding' already exists in node '2f36fe'. Skipping!
Property 'summary_embedding' already exists in node '764564'. Skipping!
Property 'summary_embedding' already exists in node 'ace78b'. Skipping!
Property 'summary_embedding' already exists in node 'e0abd2'. Skipping!
Property 'summary_embedding' already exists in node '6bbf1e'. Skipping!


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/201 [00:00<?, ?it/s]

In [37]:
df = testset.to_pandas()
df.tail(5)

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,[Those who have repented of their sins and rec...,[<1-hop>\n\nIdeas for Study and Application (P...,The teachings in 3 Nephi 11 and 3 Nephi 28:10 ...,3 Nephi 11 is significant in the context of th...,multi_hop_specific_query_synthesizer
197,How does Ezra Taft Benson's emphasis on work a...,[“In a world ever more dangerous . . . the Boo...,"[<1-hop>\n\nWork, work, work—there is no satis...",Ezra Taft Benson's emphasis on work highlights...,Ezra Taft Benson emphasized the importance of ...,multi_hop_specific_query_synthesizer
198,How can the teachings in 1 Nephi 13:39 and 1 N...,[## The Message of the Restoration: The Founda...,[<1-hop>\n\nScripture Study (Page 22) How shou...,1 Nephi 13:39 can be used in missionary work t...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...,multi_hop_specific_query_synthesizer
199,How do D&C 84:85 and D&C 68:25 guide missionar...,[#### Scripture Study (Page 33)\nWhat do the s...,[<1-hop>\n\nThe Power and Authority of Your Ca...,D&C 84:85 and D&C 68:25 guide missionaries in ...,D&C 84:85 guides missionaries by emphasizing t...,multi_hop_specific_query_synthesizer
200,How does D&C 42:12 emphasize the importance of...,[| Scriptures | Principles\n| --- | --- |\n| D...,[<1-hop>\n\nUse the Scriptures (Page 194) The ...,D&C 42:12 emphasizes the importance of using s...,D&C 42:12 emphasizes the importance of using s...,multi_hop_specific_query_synthesizer


In [14]:
df.reset_index(drop=True, inplace=True)
df.to_csv(f"../../processed_data/raw/combined_qa_pair_2.csv", index=False)

## Data Clean-up

In [15]:
dataset = pd.read_csv("../../processed_data/raw/combined_qa_pair_2.csv")
dataset.tail()

,user_input,reference_contexts,reference,synthesizer_name
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,['<1-hop>\n\nIdeas for Study and Application (...,3 Nephi 11 is significant in the context of th...,multi_hop_specific_query_synthesizer
197,How does Ezra Taft Benson's emphasis on work a...,"['<1-hop>\n\nWork, work, work—there is no sati...",Ezra Taft Benson emphasized the importance of ...,multi_hop_specific_query_synthesizer
198,How can the teachings in 1 Nephi 13:39 and 1 N...,['<1-hop>\n\nScripture Study (Page 22) How sho...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...,multi_hop_specific_query_synthesizer
199,How do D&C 84:85 and D&C 68:25 guide missionar...,['<1-hop>\n\nThe Power and Authority of Your C...,D&C 84:85 guides missionaries by emphasizing t...,multi_hop_specific_query_synthesizer
200,How does D&C 42:12 emphasize the importance of...,['<1-hop>\n\nUse the Scriptures (Page 194) The...,D&C 42:12 emphasizes the importance of using s...,multi_hop_specific_query_synthesizer


#### drop rows with NA and duplicates values

In [38]:
df = dataset.drop_duplicates(subset=["user_input", "reference_contexts", "reference"])
df.tail()

,user_input,reference_contexts,reference,synthesizer_name
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,['<1-hop>\n\nIdeas for Study and Application (...,3 Nephi 11 is significant in the context of th...,multi_hop_specific_query_synthesizer
197,How does Ezra Taft Benson's emphasis on work a...,"['<1-hop>\n\nWork, work, work—there is no sati...",Ezra Taft Benson emphasized the importance of ...,multi_hop_specific_query_synthesizer
198,How can the teachings in 1 Nephi 13:39 and 1 N...,['<1-hop>\n\nScripture Study (Page 22) How sho...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...,multi_hop_specific_query_synthesizer
199,How do D&C 84:85 and D&C 68:25 guide missionar...,['<1-hop>\n\nThe Power and Authority of Your C...,D&C 84:85 guides missionaries by emphasizing t...,multi_hop_specific_query_synthesizer
200,How does D&C 42:12 emphasize the importance of...,['<1-hop>\n\nUse the Scriptures (Page 194) The...,D&C 42:12 emphasizes the importance of using s...,multi_hop_specific_query_synthesizer


In [39]:
columns = ["user_input", "reference_contexts", "reference"]
df = df[columns]
df.head()

,user_input,reference_contexts,reference
0,Wht is the significance of Moses 1:39 in missi...,['## Preach My Gospel (D&C 50:14) (Page 5) | C...,Moses 1:39 is significant in missionary work a...
1,How can missionaries help others come unto Chr...,['The Missionary Lessons (Page 9) You have the...,Missionaries can help others come unto Christ ...
2,Wht is the role of 'Jesus the Christ' in perso...,"['Personal Study, Companion Study, District Me...","In personal study, 'Jesus the Christ' is part ..."
3,What districts do in mission work?,['### Zone Conferences (Page 11) Zone conferen...,Many missionaries serve in branches and distri...
4,How can 'Preach My Gospel' be utilized by ward...,['Use by Ward Missionaries and Leaders (Page 1...,'Preach My Gospel' is primarily for the full-t...


### drop rows with NA values


In [40]:
len(df)

201

In [41]:
df = df.dropna()
df.tail()

,user_input,reference_contexts,reference
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,['<1-hop>\n\nIdeas for Study and Application (...,3 Nephi 11 is significant in the context of th...
197,How does Ezra Taft Benson's emphasis on work a...,"['<1-hop>\n\nWork, work, work—there is no sati...",Ezra Taft Benson emphasized the importance of ...
198,How can the teachings in 1 Nephi 13:39 and 1 N...,['<1-hop>\n\nScripture Study (Page 22) How sho...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...
199,How do D&C 84:85 and D&C 68:25 guide missionar...,['<1-hop>\n\nThe Power and Authority of Your C...,D&C 84:85 guides missionaries by emphasizing t...
200,How does D&C 42:12 emphasize the importance of...,['<1-hop>\n\nUse the Scriptures (Page 194) The...,D&C 42:12 emphasizes the importance of using s...


In [43]:
# Filter out rows where 'ground_truth' contains the unwanted phrase
df_filtered = df[~df['reference'].str.contains("The answer to given question is not present in| As an AI|The context does not provide|I'm sorry,|The text doesn't provide information")]

# Display the filtered DataFrame
df_filtered.head()

,user_input,reference_contexts,reference
0,Wht is the significance of Moses 1:39 in missi...,['## Preach My Gospel (D&C 50:14) (Page 5) | C...,Moses 1:39 is significant in missionary work a...
1,How can missionaries help others come unto Chr...,['The Missionary Lessons (Page 9) You have the...,Missionaries can help others come unto Christ ...
2,Wht is the role of 'Jesus the Christ' in perso...,"['Personal Study, Companion Study, District Me...","In personal study, 'Jesus the Christ' is part ..."
3,What districts do in mission work?,['### Zone Conferences (Page 11) Zone conferen...,Many missionaries serve in branches and distri...
4,How can 'Preach My Gospel' be utilized by ward...,['Use by Ward Missionaries and Leaders (Page 1...,'Preach My Gospel' is primarily for the full-t...


In [44]:
len(df_filtered)

201

In [45]:
df_filtered.to_csv(f"../../processed_data/processed/qa_pairs.csv", index=False)

### Evaluating the baseline with Ragas

In [29]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv, find_dotenv



# Load environment variables when running locally
# Uncomment the next line if you have a .env file with your OpenAI API key
_ = load_dotenv(find_dotenv())


documents = SimpleDirectoryReader(input_files=["../../data/Pmg_lds.md", "../../data/handbook.md"]).load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()



In [32]:
from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
)

# init metrics with evaluator LLM
from ragas.llms import LlamaIndexLLMWrapper

evaluator_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o"))
metrics = [
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
]

In [33]:
# convert to Ragas Evaluation Dataset
ragas_dataset = testset.to_evaluation_dataset()
ragas_dataset

EvaluationDataset(features=['user_input', 'reference_contexts', 'reference'], len=201)

In [34]:
from ragas.integrations.llama_index import evaluate

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ragas_dataset,
)

Running Query Engine:   0%|          | 0/201 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/804 [00:00<?, ?it/s]

In [35]:
result.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_precision,context_recall
0,Wht is the significance of Moses 1:39 in missi...,[. . mine elders to be moving swiftly . . . wh...,[## Preach My Gospel (D&C 50:14) (Page 5) | Ch...,Moses 1:39 emphasizes the importance of God's ...,Moses 1:39 is significant in missionary work a...,0.571429,0.951846,0.0,0.0
1,How can missionaries help others come unto Chr...,"[Next, answer each question as though you were...",[The Missionary Lessons (Page 9) You have the ...,Missionaries can help others come unto Christ ...,Missionaries can help others come unto Christ ...,1.000000,0.980402,1.0,0.5
2,Wht is the role of 'Jesus the Christ' in perso...,[- Identify an attribute you wish to study. Fo...,"[Personal Study, Companion Study, District Mee...",The role of 'Jesus the Christ' in personal stu...,"In personal study, 'Jesus the Christ' is part ...",1.000000,0.993469,0.0,0.0
3,What districts do in mission work?,[### District Meetings and Zone Conferences (P...,[### Zone Conferences (Page 11) Zone conferenc...,Districts in mission work involve reading diff...,Many missionaries serve in branches and distri...,1.000000,0.958170,0.0,1.0
4,How can 'Preach My Gospel' be utilized by ward...,[## Use by Ward Missionaries and Leaders (Page...,[Use by Ward Missionaries and Leaders (Page 13...,Ward missionaries and leaders can utilize 'Pre...,'Preach My Gospel' is primarily for the full-t...,0.666667,0.966533,1.0,1.0
...,...,...,...,...,...,...,...,...,...
196,How do the teachings in 3 Nephi 11 and 3 Nephi...,[Those who have repented of their sins and rec...,[<1-hop>\n\nIdeas for Study and Application (P...,The teachings in 3 Nephi 11 and 3 Nephi 28:10 ...,3 Nephi 11 is significant in the context of th...,0.000000,0.993707,1.0,1.0
197,How does Ezra Taft Benson's emphasis on work a...,[“In a world ever more dangerous . . . the Boo...,"[<1-hop>\n\nWork, work, work—there is no satis...",Ezra Taft Benson's emphasis on work highlights...,Ezra Taft Benson emphasized the importance of ...,0.727273,0.941070,1.0,0.2
198,How can the teachings in 1 Nephi 13:39 and 1 N...,[## The Message of the Restoration: The Founda...,[<1-hop>\n\nScripture Study (Page 22) How shou...,1 Nephi 13:39 can be used in missionary work t...,The teachings in 1 Nephi 13:39 and 1 Nephi 1 c...,0.777778,0.950050,1.0,0.8
199,How do D&C 84:85 and D&C 68:25 guide missionar...,[#### Scripture Study (Page 33)\nWhat do the s...,[<1-hop>\n\nThe Power and Authority of Your Ca...,D&C 84:85 and D&C 68:25 guide missionaries in ...,D&C 84:85 guides missionaries by emphasizing t...,0.000000,0.969814,0.0,0.0


In [36]:
print(result)

{'faithfulness': 0.6459, 'answer_relevancy': 0.9645, 'context_precision': 0.7065, 'context_recall': 0.6481}


## run qa_pair for handbook

In [ ]:
# %pip install --upgrade "huggingface_hub>=0.20.0"

from ragas.testset import TestsetGenerator
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


documents = SimpleDirectoryReader("../../data").load_data()
# generator with openai models
generator_llm = OpenAI(model="gpt-4.1-nano")
embeddings = OpenAIEmbedding(model="text-embedding-3-large")

generator = TestsetGenerator.from_llama_index(
    llm=generator_llm,
    embedding_model=embeddings,
)

In [ ]:
# generate testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    testset_size=100,
)

In [ ]:
df = testset.to_pandas()
df

In [ ]:
df.reset_index(drop=True, inplace=True)
df.to_csv(f"../../processed_data/raw/handbook.csv", index=False)

#### remove duplicate

In [ ]:
import pandas as pd
dataset = pd.read_csv(f"./dataset.csv")
dataset.head()

In [ ]:
df_cleaned = dataset.drop_duplicates(subset=['user_input'])
df_cleaned.head()

In [ ]:
len(df_cleaned)

In [ ]:
# Select only the 'question' and 'ground_truth' columns
df_selected = df_cleaned[['user_input', 'reference_contexts', 'reference']]

# Display the selected columns
df_selected.head()

In [ ]:
# Filter out rows where 'ground_truth' contains the unwanted phrase
df_filtered = df_selected[~df_selected['reference'].str.contains("The answer to given question is not present in| As an AI|The context does not provide|I'm sorry,|The text doesn't provide information")]

# Display the filtered DataFrame
df_filtered.head()

In [ ]:
len(df_filtered)

In [ ]:
df_filtered.to_csv(f"./dataset_filtered.csv", index=False)
